In [ ]:
using CSV, DataFrames

In [ ]:
root = dirname(@__FILE__)
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")

In [ ]:
AIS = CSV.read(inputAIS, DataFrame; header=true)
AIS[8157, [1, 2, 3, 14, 25, 26]]

In [ ]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 14, 25, 26]]